In [1]:
import cvlib as cv
import cv2
import numpy as np
from cvlib.object_detection import draw_bbox
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import ImageFont, ImageDraw, Image
from playsound import playsound
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import pandas as pd
import xmltodict
import json
import datetime
import kakao_utils

def corona_open_api():
    today = datetime.datetime.today()
    base_date = today.strftime("%Y%m%d")
    base_date1 = int(base_date)

    key='U%2Bue3nwitwKpRE26tUkRgTj5%2BD2bknqcqqsDwm35y%2B3YhGcrEA%2FSYmTJn1CyrxKcPHdd5sJWJIY8PAV%2BwJxQGg%3D%3D'
    url = f'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey=U%2Bue3nwitwKpRE26tUkRgTj5%2BD2bknqcqqsDwm35y%2B3YhGcrEA%2FSYmTJn1CyrxKcPHdd5sJWJIY8PAV%2BwJxQGg%3D%3D&'
    queryParams = urlencode({ quote_plus('pageNo') : 1, 
                          quote_plus('numOfRows') : 10,
                          quote_plus('startCreateDt') : base_date1,
                          quote_plus('endCreateDt') : base_date1})

    response = urlopen(url + queryParams) 
    results = response.read().decode("utf-8")
    url2 = url + queryParams
    response = urlopen(url2)
    results = response.read().decode("utf-8")
    results_to_json = xmltodict.parse(results)
    data = json.loads(json.dumps(results_to_json))
    stateDt = (data["response"]["body"]["items"]['item']["stateDt"]) #날짜
    decideCnt = (data["response"]["body"]["items"]['item']["decideCnt"]) #확진자수
    deathCnt = (data["response"]["body"]["items"]['item']["deathCnt"]) #사망자수
    KAKAO_TOKEN_FILENAME = "C:/Users/gram/Desktop/파이썬메시업프로젝트/res/kakao_message/kakao_token.json"
    KAKAO_APP_KEY = "ec2080a0764544d8c438ffa77c95bd41"
    kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)
    corona_url = "http://ncov.mohw.go.kr"
    # 날씨 정보 만들기

    text =  f"""\
    #날짜 : {stateDt} \n
    누적확진자 : {decideCnt} \n
    사망자 수 : {deathCnt} \n
    """

    # 텍스트 템플릿 형식 만들기
    template = {
        "object_type": "text",
        "text": text,
        "link": {
            "web_url": corona_url,
            "mobile_web_url": corona_url
        },
        "button_title": "코로나 정보 상세보기"
    }
    # 카카오톡 메시지 전송
    res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)

model = load_model('C:/Users/gram/Desktop/파이썬메시업프로젝트/mask_detector.model.h5')
model.summary()
 
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()
    
while webcam.isOpened():
    status, frame = webcam.read()
    
    if not status:
        print("Could not read frame")
        exit()

    faces, confidences = cv.detect_face(frame)

# face로 detect된 faces에서 얼굴 하나씩 좌표 저장
    for idx, face in enumerate(faces):
    # detect한 얼굴 표시하기 위해 사진 위에 얼굴 부분 사각형 그리기 (green으로 두께는 2)
        (startX,startY) = face[0],face[1]
        (endX,endY) = face[2],face[3]

        if 0 <= startX <= frame.shape[1] and 0 <= endX <= frame.shape[1] and 0 <= startY <= frame.shape[0] and 0 <= endY <= frame.shape[0]:
            
            face_region = frame[startY:endY, startX:endX]
            
            face_region1 = cv2.resize(face_region, (224, 224), interpolation = cv2.INTER_AREA)
            
            x = img_to_array(face_region1)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            
            prediction = model.predict(x)
            #print(prediction)
            
            if np.any(prediction < 0.5): 
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "Mask ({:.2f}%)".format((1 - prediction[0][0])*100)
                cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                                
            else:
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,0,255), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "No Mask ({:.2f}%)".format(prediction[0][0]*100)
                cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                playsound('C:/Users/gram/Desktop/마스크/c_lt_kyuri_2.1.13_414-nvoice_kyuri_2.1.mp3')
                corona_open_api()

    cv2.imshow("mask nomask classify", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cv2.destroyAllWindows() 


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
flatten (Flatten)            (None, 62720)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               16056576  
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 62)                7998      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 63        
Total params: 18,355,517
Trainable params: 16,097,533
Non-trainable params: 2,257,984
____________________________________